## **Measuring the Impact of Transit Access on Unemployment across Canada**

#### **An Analysis of Major Canadian Cities in 2023 and 2024**

## ***clean_transit_data.py***

In [20]:
import pandas as pd

parquet_path = "data/01-raw_data/public_transport_access.parquet"
transit_data = pd.read_parquet(parquet_path)

Functions to clean and preprocess data for Canadian cities.

In [21]:
def print_unique_values(df):
    """Prints unique values for each column in a DataFrame, showing up to 10 for columns with many unique values."""
    for col in df.columns:
        unique_vals = df[col].unique()
        num_unique = len(unique_vals)
        print(f"--- {col} ({num_unique} unique values) ---")
        if num_unique < 15:
            print(unique_vals)
        else:
            print(unique_vals[:10])
            print(f"... and {num_unique - 10} more")
        print("\n")

In [22]:
def check_id_consistency(df1, df2, id_col1, id_col2, name_col1=None, name_col2=None, label1="Dataset 1", label2="Dataset 2"):
    """
    Compares unique identifiers between two DataFrames to check for consistency.

    Args:
        df1, df2: The DataFrames to compare.
        id_col1, id_col2: The names of the ID columns to match on.
        name_col1, name_col2: (Optional) Descriptive columns to display alongside IDs. 
                              If None, only IDs are displayed.
        label1, label2: (Optional) String labels for the print output.
    """
    
    # 1. Get unique IDs as sets
    ids_1 = set(df1[id_col1].unique())
    ids_2 = set(df2[id_col2].unique())

    # 2. Find intersection and differences
    common_ids = ids_1 & ids_2
    only_in_1 = ids_1 - ids_2
    only_in_2 = ids_2 - ids_1
    
    # Helper to select columns for display
    cols_1 = [id_col1, name_col1] if name_col1 else [id_col1]
    cols_2 = [id_col2, name_col2] if name_col2 else [id_col2]

    print(f"--- COMPARISON: {label1} vs. {label2} ---\n")

    # 3. Only in Dataset 1
    print(f"IDs only in {label1}: ({len(only_in_1)})")
    if only_in_1:
        display_df = df1[df1[id_col1].isin(only_in_1)][cols_1].drop_duplicates()
        print(display_df.to_string(index=False))
    else:
        print("None")
    print("-" * 40)

    # 4. Only in Dataset 2
    print(f"IDs only in {label2}: ({len(only_in_2)})")
    if only_in_2:
        display_df = df2[df2[id_col2].isin(only_in_2)][cols_2].drop_duplicates()
        print(display_df.to_string(index=False))
    else:
        print("None")
    print("-" * 40)

    # 5. Common to both
    print(f"IDs present in BOTH: ({len(common_ids)})")
    if common_ids:
        # Defaults to showing descriptive info from DF1
        display_df = df1[df1[id_col1].isin(common_ids)][cols_1].drop_duplicates()
        print(display_df.head(10).to_string(index=False)) # Limiting to 10 rows for readability
        if len(common_ids) > 10:
            print(f"...and {len(common_ids) - 10} more.")
    else:
        print("None")

In [23]:
# 1Identify missing values in GEO
missing_mask = transit_data['VALUE'].isna()

# Get the list of cities (GEOs) that have at least one missing value
cmas_to_drop = transit_data.loc[missing_mask, 'GEO'].unique()
print("CMAs with missing values in VALUE:", cmas_to_drop)

# The identified CMA (Saguenay) has all missing values for VALUE, so we will drop it entirely
transit_data = transit_data[~transit_data['GEO'].isin(cmas_to_drop)].copy()

CMAs with missing values in VALUE: ['Saguenay, Census metropolitan area (CMA)']


In [24]:
cols_to_keep = [
    "GEO",
    "REF_DATE",
    "DGUID",
    "Distance-capacity public transit service area",
    "Demographic and socio-economic",
    "Sustainable Development Goals (SDGs) 11.2.1 indicator",
    "UOM",
    "VALUE"
]
transit_data = transit_data[cols_to_keep].copy()

# Rename columns for clarity
transit_data = transit_data.rename(columns={
    "GEO": "CMA",
    "REF_DATE": "Year",
    "DGUID": "CMA_ID",
    "Distance-capacity public transit service area": "Transit_Distance_Category",
    "Demographic and socio-economic": "Transit_Profile_Characteristic",
    "Sustainable Development Goals (SDGs) 11.2.1 indicator": "Measure",
    "UOM": "Transit_Unit_of_Measure",
    "VALUE": "Transit_Value"
})
transit_data.head()

# Convert 'CMA' to string, remove ", Census metropolitan area (CMA)" to make it cleaner
transit_data['CMA'] = transit_data['CMA'].str.replace(", Census metropolitan area (CMA)", "", regex=False)

In [25]:
print_unique_values(transit_data)
transit_data

--- CMA (41 unique values) ---
["St. John's" 'Halifax' 'Fredericton' 'Moncton' 'Saint John'
 'Drummondville' 'Montréal' 'Ottawa - Gatineau (Quebec part)' 'Québec'
 'Sherbrooke']
... and 31 more


--- Year (2 unique values) ---
[2023 2024]


--- CMA_ID (41 unique values) ---
['2021S0503001' '2021S0503205' '2021S0503320' '2021S0503305'
 '2021S0503310' '2021S0503447' '2021S0503462' '2021S050524505'
 '2021S0503421' '2021S0503433']
... and 31 more


--- Transit_Distance_Category (4 unique values) ---
['400 metres from all public transit stops'
 '400 metres from low-capacity public transit stops only'
 '500 metres from all public transit stops'
 '500 metres from low-capacity public transit stops only']


--- Transit_Profile_Characteristic (9 unique values) ---
['Total - Age groups of the population - 100% data' '0 to 14 years'
 '15 to 64 years' '65 years and over'
 'Total - Population aged 15 years and over by labour force status - 25% sample data'
 'In the labour force' 'Employed' 'Unemploy

,CMA,Year,CMA_ID,Transit_Distance_Category,Transit_Profile_Characteristic,Measure,Transit_Unit_of_Measure,Transit_Value
0,St. John's,2023,2021S0503001,400 metres from all public transit stops,Total - Age groups of the population - 100% data,Count of population within service area,Persons,105010.0
1,St. John's,2024,2021S0503001,400 metres from all public transit stops,Total - Age groups of the population - 100% data,Count of population within service area,Persons,104585.0
2,St. John's,2023,2021S0503001,400 metres from all public transit stops,Total - Age groups of the population - 100% data,Proportion of population within service area,Percent,49.4
3,St. John's,2024,2021S0503001,400 metres from all public transit stops,Total - Age groups of the population - 100% data,Proportion of population within service area,Percent,49.2
4,St. John's,2023,2021S0503001,400 metres from all public transit stops,0 to 14 years,Count of population within service area,Persons,12945.0
...,...,...,...,...,...,...,...,...
6043,Victoria,2024,2021S0503935,500 metres from low-capacity public transit st...,Unemployed,Proportion of population within service area,Percent,87.5
6044,Victoria,2023,2021S0503935,500 metres from low-capacity public transit st...,Not in the labour force,Count of population within service area,Persons,102475.0
6045,Victoria,2024,2021S0503935,500 metres from low-capacity public transit st...,Not in the labour force,Count of population within service area,Persons,102850.0
6046,Victoria,2023,2021S0503935,500 metres from low-capacity public transit st...,Not in the labour force,Proportion of population within service area,Percent,83.2


## ***clean_labour_data.py***

In [26]:
import pandas as pd

parquet_path = "data/01-raw_data/labour_rates.parquet"
labour_data = pd.read_parquet(parquet_path)

In [27]:
check_id_consistency(
    df1=transit_data,
    df2=labour_data,
    id_col1="CMA_ID",
    id_col2="DGUID",
    name_col1="CMA",
    name_col2="GEO",
    label1="Transit Data",
    label2="Labour Data")

--- COMPARISON: Transit Data vs. Labour Data ---

IDs only in Transit Data: (1)
      CMA_ID        CMA
2021S0503001 St. John's
----------------------------------------
IDs only in Labour Data: (3)
       DGUID                                   GEO
  2021S05031 St. John's, Newfoundland and Labrador
2021S0503408                      Saguenay, Quebec
2021S0503505       Ottawa-Gatineau, Ontario/Quebec
----------------------------------------
IDs present in BOTH: (40)
        CMA_ID                             CMA
  2021S0503205                         Halifax
  2021S0503320                     Fredericton
  2021S0503305                         Moncton
  2021S0503310                      Saint John
  2021S0503447                   Drummondville
  2021S0503462                        Montréal
2021S050524505 Ottawa - Gatineau (Quebec part)
  2021S0503421                          Québec
  2021S0503433                      Sherbrooke
  2021S0503442                  Trois-Rivières
...and 30 more

To ensure consistency between datasets, we identified that the DGUID ‘2021S05031’ in the labour data corresponded to St. John’s, matching the CMA_ID ‘2021S0503001’ in the transit data. Based on this partial match in city names, we manually replaced the DGUID in the labour data to enable accurate merging.

In [28]:
# Replace DGUID value in labour_data
labour_data['DGUID'] = labour_data['DGUID'].replace('2021S05031', '2021S0503001')

No data was found for Saguenay, Quebec (DGUID ‘2021S0503408’) in the transit dataset, so this city was excluded from the analysis. 

For Ottawa-Gatineau, Ontario/Quebec (DGUID ‘2021S0503505’), the city is split into Ontario and Quebec parts in the transit data, which are already represented as separate entries; therefore, the combined DGUID was not used.

We drop this DGUID from the labour data to maintain consistency across datasets.


In [29]:
to_drop = ['2021S0503408', '2021S0503505']
labour_data = labour_data[~labour_data['DGUID'].isin(to_drop)].copy()

In [30]:
# Get unique IDs from each DataFrame
transit_ids = set(transit_data["CMA_ID"].unique())
labour_ids = set(labour_data["DGUID"].unique())

# Find common and exclusive IDs
common_ids = transit_ids & labour_ids
only_in_transit = transit_ids - labour_ids
only_in_labour = labour_ids - transit_ids

print(f"Common CMA_IDs: {len(common_ids)}")
print(f"Only in transit_data: {len(only_in_transit)}")
print(f"Only in labour_data: {len(only_in_labour)}")

Common CMA_IDs: 41
Only in transit_data: 0
Only in labour_data: 0


In [31]:
check_id_consistency(
    df1=transit_data,
    df2=labour_data,
    id_col1="CMA_ID",
    id_col2="DGUID",
    name_col1="CMA",
    name_col2="GEO",
    label1="Transit Data",
    label2="Labour Data")

--- COMPARISON: Transit Data vs. Labour Data ---

IDs only in Transit Data: (0)
None
----------------------------------------
IDs only in Labour Data: (0)
None
----------------------------------------
IDs present in BOTH: (41)
        CMA_ID                             CMA
  2021S0503001                      St. John's
  2021S0503205                         Halifax
  2021S0503320                     Fredericton
  2021S0503305                         Moncton
  2021S0503310                      Saint John
  2021S0503447                   Drummondville
  2021S0503462                        Montréal
2021S050524505 Ottawa - Gatineau (Quebec part)
  2021S0503421                          Québec
  2021S0503433                      Sherbrooke
...and 31 more.


In [32]:
cols_to_keep = [
    "REF_DATE",
    "GEO",
    "DGUID",
    "Labour force characteristics",
    "Data type",
    "UOM",
    "VALUE"
]
labour_data = labour_data[cols_to_keep].copy()

labour_data = labour_data.rename(columns={
    "REF_DATE": "Time_Period",
    "GEO": "CMA",
    "DGUID": "CMA_ID",
    "Labour force characteristics": "Labour_Metric",
    "Data type": "Labour_Data_Type",
    "UOM": "Labour_Unit_of_Measure",
    "VALUE": "Labour_Value"
})

# Create a 'Year' column by splitting the 'Time_Period' string
labour_data['Year'] = labour_data['Time_Period'].str.split('-').str[0]

labour_data.head(10)

,Time_Period,CMA,CMA_ID,Labour_Metric,Labour_Data_Type,Labour_Unit_of_Measure,Labour_Value,Year
0,2023-01,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,191.5,2023
1,2023-02,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,192.1,2023
2,2023-03,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,192.7,2023
3,2023-04,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,193.3,2023
4,2023-05,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,193.8,2023
5,2023-06,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,194.4,2023
6,2023-07,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,195.0,2023
7,2023-08,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,195.5,2023
8,2023-09,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,196.0,2023
9,2023-10,"St. John's, Newfoundland and Labrador",2021S0503001,Population,Seasonally adjusted,Persons in thousands,196.6,2023


In [33]:
sorted(transit_data['CMA_ID'].unique()) == sorted(labour_data['CMA_ID'].unique())

True

## ***clean_commute_data.py***

In [34]:
parquet_path = "data/01-raw_data/commute_times.parquet"
commute_data = pd.read_parquet(parquet_path)

In [35]:
check_id_consistency(
    df1=transit_data,
    df2=commute_data,
    id_col1="CMA_ID",
    id_col2="DGUID",
    name_col1="CMA",
    name_col2="GEO",
    label1="Transit Data",
    label2="Commute Data")

--- COMPARISON: Transit Data vs. Commute Data ---

IDs only in Transit Data: (2)
        CMA_ID                              CMA
2021S050524505  Ottawa - Gatineau (Quebec part)
2021S050535505 Ottawa - Gatineau (Ontario part)
----------------------------------------
IDs only in Commute Data: (11)
       DGUID                                    GEO
2021S0504450                   Granby(CA) 450, Que.
2021S0504850          Grande Prairie(CA) 850, Alta.
2021S0504805            Medicine Hat(CA) 805, Alta.
2021S0504575                North Bay(CA) 575, Ont.
2021S0503505 Ottawa - Gatineau (CMA) 505, Ont./Que.
2021S0504970            Prince George(CA) 970, B.C.
2021S0503408               Saguenay (CMA) 408, Que.
2021S0504452          Saint-Hyacinthe(CA) 452, Que.
2021S0504562                   Sarnia(CA) 562, Ont.
2021S0504590         Sault Ste. Marie(CA) 590, Ont.
2021S0504860            Wood Buffalo(CA) 860, Alta.
----------------------------------------
IDs present in BOTH: (39)
      CMA_ID

In [36]:
unified_dguid = '2021S0503505'
id_quebec_part = '2021S050524505'
id_ontario_part = '2021S050535505'


# --- 1. Isolate ALL rows for the unified DGUID ---
ottawa_unified_rows = commute_data[commute_data['DGUID'] == unified_dguid].copy()

# --- 2. Create the two split datasets ---

# Copy 1: Quebec Part
ottawa_quebec_part = ottawa_unified_rows.copy()
# Assign the new ID and update the GEO description
ottawa_quebec_part['DGUID'] = id_quebec_part
ottawa_quebec_part['GEO'] = 'Ottawa-Gatineau, Quebec part, Ontario/Quebec'

# Copy 2: Ontario Part
ottawa_ontario_part = ottawa_unified_rows.copy()
# Assign the new ID and update the GEO description
ottawa_ontario_part['DGUID'] = id_ontario_part
ottawa_ontario_part['GEO'] = 'Ottawa-Gatineau, Ontario part, Ontario/Quebec'

# --- 3. Combine the new split datasets ---
ottawa_split_data = pd.concat([ottawa_quebec_part, ottawa_ontario_part])

# --- 4. Remove the original rows and add the new ones ---

# Filter out the original unified DGUID rows from the main DataFrame
commute_data_filter = commute_data[commute_data['DGUID'] != unified_dguid].copy()

# Concatenate the filtered data with the newly created split data
commute_data = pd.concat([commute_data_filter, ottawa_split_data])

In [37]:
check_id_consistency(
    df1=transit_data,
    df2=commute_data,
    id_col1="CMA_ID",
    id_col2="DGUID",
    name_col1="CMA",
    name_col2="GEO",
    label1="Transit Data",
    label2="Commute Data")

--- COMPARISON: Transit Data vs. Commute Data ---

IDs only in Transit Data: (0)
None
----------------------------------------
IDs only in Commute Data: (10)
       DGUID                            GEO
2021S0504450           Granby(CA) 450, Que.
2021S0504850  Grande Prairie(CA) 850, Alta.
2021S0504805    Medicine Hat(CA) 805, Alta.
2021S0504575        North Bay(CA) 575, Ont.
2021S0504970    Prince George(CA) 970, B.C.
2021S0503408       Saguenay (CMA) 408, Que.
2021S0504452  Saint-Hyacinthe(CA) 452, Que.
2021S0504562           Sarnia(CA) 562, Ont.
2021S0504590 Sault Ste. Marie(CA) 590, Ont.
2021S0504860    Wood Buffalo(CA) 860, Alta.
----------------------------------------
IDs present in BOTH: (41)
        CMA_ID                             CMA
  2021S0503001                      St. John's
  2021S0503205                         Halifax
  2021S0503320                     Fredericton
  2021S0503305                         Moncton
  2021S0503310                      Saint John
  2021S05

In [38]:
cols_to_keep = [
    "GEO",
    "DGUID",
    "Main mode of commuting (21)",
    "Commuting duration (7)",
    "VALUE"
]
commute_data = commute_data[cols_to_keep].copy()

commute_data = commute_data.rename(columns={
    "GEO": "CMA",
    "DGUID": "CMA_ID",
    "Main mode of commuting (21)": "Commute_Mode",
    "Commuting duration (7)": "Average_Commute_Duration",
    "VALUE": "Commute_Value"
})
commute_data.head()

,CMA,CMA_ID,Commute_Mode,Average_Commute_Duration,Commute_Value
0,"Abbotsford - Mission (CMA) 932, B.C.",2021S0503932,Total - Main mode of commuting,Average commuting duration (in minutes),19.1
1,"Abbotsford - Mission (CMA) 932, B.C.",2021S0503932,"Car, truck or van",Average commuting duration (in minutes),19.2
2,"Abbotsford - Mission (CMA) 932, B.C.",2021S0503932,Sustainable transportation,Average commuting duration (in minutes),19.6
3,"Abbotsford - Mission (CMA) 932, B.C.",2021S0503932,Public transit,Average commuting duration (in minutes),33.0
4,"Barrie (CMA) 568, Ont.",2021S0503568,Total - Main mode of commuting,Average commuting duration (in minutes),20.3


We now have three datasets cleaned and ready for analysis: transit access, unemployment rates, and commute times. The next step is to merge these datasets based on the common city identifiers to conduct an analysis of how transit access impacts unemployment rates across Canadian cities.

## ***analysis_data.py***

Pivot each dataset into a wide format for analysis and merging.

In [39]:
objects_to_process = [transit_data, labour_data, commute_data]

# Apply the function print_unique_values to the 3 datasets
results = list(map(print_unique_values, objects_to_process))

--- CMA (41 unique values) ---
["St. John's" 'Halifax' 'Fredericton' 'Moncton' 'Saint John'
 'Drummondville' 'Montréal' 'Ottawa - Gatineau (Quebec part)' 'Québec'
 'Sherbrooke']
... and 31 more


--- Year (2 unique values) ---
[2023 2024]


--- CMA_ID (41 unique values) ---
['2021S0503001' '2021S0503205' '2021S0503320' '2021S0503305'
 '2021S0503310' '2021S0503447' '2021S0503462' '2021S050524505'
 '2021S0503421' '2021S0503433']
... and 31 more


--- Transit_Distance_Category (4 unique values) ---
['400 metres from all public transit stops'
 '400 metres from low-capacity public transit stops only'
 '500 metres from all public transit stops'
 '500 metres from low-capacity public transit stops only']


--- Transit_Profile_Characteristic (9 unique values) ---
['Total - Age groups of the population - 100% data' '0 to 14 years'
 '15 to 64 years' '65 years and over'
 'Total - Population aged 15 years and over by labour force status - 25% sample data'
 'In the labour force' 'Employed' 'Unemploy

### **Transit Access Data:**

In [40]:
# Filter the columns needed for transit
transit_years = [2023, 2024]

allowed_characteristics = [
    'Total - Age groups of the population - 100% data',
    '15 to 64 years',
    '65 years and over',
    'In the labour force',
    'Employed',
    'Unemployed',
    'Not in the labour force',
    'Total - Population aged 15 years and over by labour force status - 25% sample data'
]

transit_category = '500 metres from all public transit stops'

#'500 metres from low-capacity public transit stops only'
#'500 metres from all public transit stops'
#'400 metres from low-capacity public transit stops only'
#'400 metres from all public transit stops'

transit_filter = transit_data[
    (transit_data['Year'].isin(transit_years)) &
    (transit_data['Transit_Distance_Category'] == transit_category) &
    (transit_data['Transit_Profile_Characteristic'].isin(allowed_characteristics)) &
    (transit_data['Transit_Unit_of_Measure'] == 'Percent')
]
transit_filter.head()

transit_pivot = transit_filter.pivot_table(
    index=['CMA_ID', 'Transit_Distance_Category'], # Keep Year and Distance for more detail
    columns=['Transit_Profile_Characteristic', 'Year'],
    values='Transit_Value'
).reset_index()

# Flatten the multi-level column names for easier use
transit_pivot.columns = [''.join(map(str, col)).strip() for col in transit_pivot.columns.values]
transit_pivot.head()

,CMA_ID,Transit_Distance_Category,15 to 64 years2023,15 to 64 years2024,65 years and over2023,65 years and over2024,Employed2023,Employed2024,In the labour force2023,In the labour force2024,Not in the labour force2023,Not in the labour force2024,Total - Age groups of the population - 100% data2023,Total - Age groups of the population - 100% data2024,Total - Population aged 15 years and over by labour force status - 25% sample data2023,Total - Population aged 15 years and over by labour force status - 25% sample data2024,Unemployed2023,Unemployed2024
0,2021S0503001,500 metres from all public transit stops,54.8,54.7,61.0,61.0,52.9,52.6,53.3,53.1,59.8,59.8,54.7,54.5,55.7,55.5,56.7,56.7
1,2021S0503205,500 metres from all public transit stops,66.5,66.5,65.6,65.7,65.5,65.6,66.2,66.2,65.7,65.8,65.5,65.5,66.0,66.1,71.3,71.4
2,2021S0503305,500 metres from all public transit stops,63.4,62.8,65.1,64.2,62.9,62.3,63.2,62.6,64.8,64.1,63.2,62.6,63.8,63.1,66.1,65.4
3,2021S0503310,500 metres from all public transit stops,42.7,40.5,42.3,38.6,40.0,37.6,40.8,38.6,43.5,41.1,42.2,39.7,41.8,39.5,48.8,47.5
4,2021S0503320,500 metres from all public transit stops,49.0,49.0,51.0,51.0,49.3,49.3,49.7,49.7,48.3,48.3,48.5,48.5,49.2,49.2,54.6,54.6


### **Labour Data:**

In [41]:
rate_metrics = ['Unemployment rate', 'Participation rate', 'Employment rate']
labour_years = ['2023', '2024']

# Labour metrics
is_rate_percent = (
    (labour_data['Labour_Metric'].isin(rate_metrics)) & 
    (labour_data['Labour_Unit_of_Measure'] == 'Percent')
)

# Population, in 'Persons in thousands'
is_population = (
    (labour_data['Labour_Metric'] == 'Population') & 
    (labour_data['Labour_Unit_of_Measure'] == 'Persons in thousands')
)

labour_filter = labour_data[
    (is_rate_percent | is_population) &  # <--- The Magic OR
    (labour_data['Labour_Data_Type'] == 'Seasonally adjusted') &
    (labour_data['Year'].isin(labour_years))
]

labour_filter.head()

# Compute mean of the Labour_Value grouped by CMA_ID and Year
labour_aggregated = labour_filter.groupby(['CMA_ID', 'Labour_Metric', 'Year']).agg(
    Aggregated_Value=('Labour_Value', 'mean')
).reset_index()

labour_pivot = labour_aggregated.pivot_table(
    index=['CMA_ID'],
    columns=['Labour_Metric', 'Year'],
    values='Aggregated_Value'
).reset_index()

labour_pivot.columns = [''.join(map(str, col)).strip() for col in labour_pivot.columns.values]
labour_pivot.head()

,CMA_ID,Employment rate2023,Employment rate2024,Participation rate2023,Participation rate2024,Population2023,Population2024,Unemployment rate2023,Unemployment rate2024
0,2021S0503001,60.975000,60.700000,64.875000,65.216667,194.633333,201.191667,6.016667,6.891667
1,2021S0503205,64.658333,65.266667,68.458333,69.058333,426.450000,444.875000,5.558333,5.475000
2,2021S0503305,62.416667,62.208333,66.116667,65.733333,147.141667,155.975000,5.583333,5.341667
3,2021S0503310,58.350000,60.425000,61.841667,64.358333,115.700000,118.541667,5.650000,6.108333
4,2021S0503320,61.516667,61.516667,64.508333,65.191667,97.508333,101.516667,4.641667,5.666667


### **Commute Data:**

In [42]:
commute_pivot = commute_data.pivot_table(
    index='CMA_ID',
    columns='Commute_Mode',
    values='Commute_Value'
).reset_index()

# Clean up column names by adding a prefix
rename_map = {}
for col in commute_pivot.columns:
    if col != 'CMA_ID':
        rename_map[col] = f'Commute_Avg_{col}'

commute_pivot.rename(columns=rename_map, inplace=True)
print(commute_pivot.columns)

Index(['CMA_ID', 'Commute_Avg_Car, truck or van', 'Commute_Avg_Public transit',
       'Commute_Avg_Sustainable transportation',
       'Commute_Avg_Total - Main mode of commuting'],
      dtype='object', name='Commute_Mode')


### Merge all datasets on the common city identifiers.

In [38]:
# 1. Merge Commute and Transit data
first_merge = pd.merge(
    transit_pivot,
    commute_pivot,
    on='CMA_ID',
    how='left' # Keep all rows from the transit data
)

# 2. Merge with Labour data
second_merge = pd.merge(
    first_merge,
    labour_pivot,
    on='CMA_ID',
    how='left'
)

In [39]:
# Add CMA names back for clarity

cma_mapping = transit_data[['CMA_ID', 'CMA']].drop_duplicates().reset_index(drop=True)
cma_mapping = cma_mapping.rename(columns={'CMA': 'CMA_Name'})

analysis_data = pd.merge(
    second_merge,
    cma_mapping,
    on='CMA_ID',
    how='left'
)

# Reorder columns to have CMA_Name after CMA_ID

cols = analysis_data.columns.tolist()
cols.insert(1, cols.pop(cols.index('CMA_Name')))
analysis_data = analysis_data[cols]

analysis_data.head()

,CMA_ID,CMA_Name,Transit_Distance_Category,15 to 64 years2023,15 to 64 years2024,65 years and over2023,65 years and over2024,Employed2023,Employed2024,In the labour force2023,...,Commute_Avg_Sustainable transportation,Commute_Avg_Total - Main mode of commuting,Employment rate2023,Employment rate2024,Participation rate2023,Participation rate2024,Population2023,Population2024,Unemployment rate2023,Unemployment rate2024
0,2021S0503001,St. John's,500 metres from all public transit stops,54.8,54.7,61.0,61.0,52.9,52.6,53.3,...,19.7,17.0,60.975000,60.700000,64.875000,65.216667,194.633333,201.191667,6.016667,6.891667
1,2021S0503205,Halifax,500 metres from all public transit stops,66.5,66.5,65.6,65.7,65.5,65.6,66.2,...,25.2,21.4,64.658333,65.266667,68.458333,69.058333,426.450000,444.875000,5.558333,5.475000
2,2021S0503305,Moncton,500 metres from all public transit stops,63.4,62.8,65.1,64.2,62.9,62.3,63.2,...,19.1,16.8,62.416667,62.208333,66.116667,65.733333,147.141667,155.975000,5.583333,5.341667
3,2021S0503310,Saint John,500 metres from all public transit stops,42.7,40.5,42.3,38.6,40.0,37.6,40.8,...,17.0,19.0,58.350000,60.425000,61.841667,64.358333,115.700000,118.541667,5.650000,6.108333
4,2021S0503320,Fredericton,500 metres from all public transit stops,49.0,49.0,51.0,51.0,49.3,49.3,49.7,...,18.1,17.3,61.516667,61.516667,64.508333,65.191667,97.508333,101.516667,4.641667,5.666667


### Polish the analysis dataset with clear column names.

In [40]:
# Regex explanation: Replace anything that IS NOT (^) alphanumeric or underscore with nothing
analysis_data.columns = analysis_data.columns.str.replace(r'[^a-zA-Z0-9_]', '', regex=True)

analysis_data.columns = [
    f"Col_{c}" if c[0].isdigit() else c 
    for c in analysis_data.columns
]
print_unique_values(analysis_data)

--- CMA_ID (41 unique values) ---
['2021S0503001' '2021S0503205' '2021S0503305' '2021S0503310'
 '2021S0503320' '2021S0503421' '2021S0503433' '2021S0503442'
 '2021S0503447' '2021S0503462']
... and 31 more


--- CMA_Name (41 unique values) ---
["St. John's" 'Halifax' 'Moncton' 'Saint John' 'Fredericton' 'Québec'
 'Sherbrooke' 'Trois-Rivières' 'Drummondville' 'Montréal']
... and 31 more


--- Transit_Distance_Category (1 unique values) ---
['500 metres from all public transit stops']


--- Col_15to64years2023 (40 unique values) ---
[54.8 66.5 63.4 42.7 49.  79.8 72.5 75.3 34.6 87.1]
... and 30 more


--- Col_15to64years2024 (41 unique values) ---
[54.7 66.5 62.8 40.5 49.  79.8 64.5 73.6 55.6 85.9]
... and 31 more


--- Col_65yearsandover2023 (40 unique values) ---
[61.  65.6 65.1 42.3 51.  85.3 77.6 79.5 50.5 89.8]
... and 30 more


--- Col_65yearsandover2024 (38 unique values) ---
[61.  65.7 64.2 38.6 51.  85.4 76.9 63.7 87.8 66. ]
... and 28 more


--- Employed2023 (39 unique values) --

In [41]:
print("Final analysis_data columns:")
for col in analysis_data.columns:
    print(col)

Final analysis_data columns:
CMA_ID
CMA_Name
Transit_Distance_Category
Col_15to64years2023
Col_15to64years2024
Col_65yearsandover2023
Col_65yearsandover2024
Employed2023
Employed2024
Inthelabourforce2023
Inthelabourforce2024
Notinthelabourforce2023
Notinthelabourforce2024
TotalAgegroupsofthepopulation100data2023
TotalAgegroupsofthepopulation100data2024
TotalPopulationaged15yearsandoverbylabourforcestatus25sampledata2023
TotalPopulationaged15yearsandoverbylabourforcestatus25sampledata2024
Unemployed2023
Unemployed2024
Commute_Avg_Cartruckorvan
Commute_Avg_Publictransit
Commute_Avg_Sustainabletransportation
Commute_Avg_TotalMainmodeofcommuting
Employmentrate2023
Employmentrate2024
Participationrate2023
Participationrate2024
Population2023
Population2024
Unemploymentrate2023
Unemploymentrate2024


In [ ]:
import numpy as np
# Create Log Population Variable
analysis_data['Log_Pop_2024'] = np.log(analysis_data['Population2024'])

# Create Transit Penalty Variable (if not already created)
analysis_data['Transit_Penalty'] = analysis_data['Commute_Avg_Publictransit'] - model_data['Commute_Avg_Cartruckorvan']

In [42]:
csv_path = "data/02-analysis_data/analysis_data.csv"
parquet_path = "data/02-analysis_data/analysis_data.parquet"

analysis_data.to_csv(csv_path, index=False)
analysis_data.to_parquet(parquet_path)

## ***model_data.py***

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

model_data = analysis_data.copy()

# Formula: Participation ~ Access + Efficiency (Penalty) + City Size (Log Pop)
formula_final = (
    'Participationrate2024 ~ '
    'Col_15to64years2024 + '      # Access
    'Transit_Penalty + '          # Efficiency (Testing the bus vs car gap)
    'Log_Pop_2024'                # Population Size
)

model_final = smf.ols(formula=formula_final, data=model_data).fit()
print("REGRESSION MODEL: YEAR 2024")
print(model_final.summary())

REGRESSION MODEL: YEAR 2024
                              OLS Regression Results                             
Dep. Variable:     Participationrate2024   R-squared:                       0.553
Model:                               OLS   Adj. R-squared:                  0.517
Method:                    Least Squares   F-statistic:                     15.26
Date:                   Tue, 16 Dec 2025   Prob (F-statistic):           1.28e-06
Time:                           08:15:49   Log-Likelihood:                -100.86
No. Observations:                     41   AIC:                             209.7
Df Residuals:                         37   BIC:                             216.6
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

In [ ]:
model_data = analysis_data.copy()

model_data['Log_Pop_2023'] = np.log(model_data['Population2023'])
model_data['Transit_Penalty'] = model_data['Commute_Avg_Publictransit'] - model_data['Commute_Avg_Cartruckorvan']

# Formula: Participation ~ Access + Efficiency (Penalty) + City Size (Log Pop)
formula_final = (
    'Participationrate2023 ~ '
    'Col_15to64years2023 + '      # Access
    'Transit_Penalty + '          # Efficiency (Testing the bus vs car gap)
    'Log_Pop_2023'                # Population Size
)

model_final = smf.ols(formula=formula_final, data=model_data).fit()
print("REGRESSION MODEL: YEAR 2023")
print(model_final.summary())

REGRESSION MODEL: YEAR 2023
                              OLS Regression Results                             
Dep. Variable:     Participationrate2023   R-squared:                       0.391
Model:                               OLS   Adj. R-squared:                  0.359
Method:                    Least Squares   F-statistic:                     12.20
Date:                   Tue, 16 Dec 2025   Prob (F-statistic):           8.08e-05
Time:                           08:06:00   Log-Likelihood:                -97.999
No. Observations:                     41   AIC:                             202.0
Df Residuals:                         38   BIC:                             207.1
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------